# Missing values imputation fun using ML.
## Steps:
1. Import necessary libraries
2. Load data
3. Find the kolumns with missing values and store in an objekt.
4. Find the kolumns based on data type.
   1. numerik
   2. Categories
   3. Boolean
5. Define the fun to impute missing values
6. Apply the fun to dataset with missing values
7. Check the missing values after imputation

In [67]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, precision_score, r2_score

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [68]:
# load data set
df = pd.read_csv('heart_disease_uci.csv')
df.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [69]:
df.isnull().sum()[df.isnull().sum() > 0].sort_values(ascending = False)

ca          611
thal        486
slope       309
fbs          90
oldpeak      62
trestbps     59
thalch       55
exang        55
chol         30
restecg       2
dtype: int64

In [70]:
df.isnull().sum()[df.isnull().sum() > 0].index.tolist()

['trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalch',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal']

In [71]:
missing_data_kols = df.isnull().sum()[df.isnull().sum() > 0].index.tolist()
missing_data_kols

['trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalch',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal']

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB


# Segregating kategorikal and numerikal kolumns

In [73]:
kat_kols =  df.select_dtypes(include='object').columns.tolist()
kat_kols

['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

In [74]:
bool_kols =  df.select_dtypes(include='bool').columns.tolist()
bool_kols

[]

In [75]:
numerik_kols =  df.select_dtypes(exclude='object').columns.tolist()
numerik_kols

['id', 'age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca', 'num']

In [76]:
print(f'kat_kols: {kat_kols}')
print(f'numerik_kols: {numerik_kols}')

kat_kols: ['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']
numerik_kols: ['id', 'age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca', 'num']


In [77]:
missing_data_kols = df.isnull().sum()[df.isnull().sum() > 0].index.tolist()
missing_data_kols

['trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalch',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal']

# defina a fun to impute the missing values in that kolumn, whether the kolumn is kategorikal or numerikal. Fun should also return metriks of ML model.

In [83]:
df = pd.read_csv('heart_disease_uci.csv')

In [84]:
missing_data_kols = df.isnull().sum()[df.isnull().sum() > 0].index.tolist()
missing_data_kols

['trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalch',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal']

In [ ]:
import pandas as pd
import warnings
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split

# Suppress specific warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=UserWarning)  # General suppression for other warnings

def impute_and_evaluate(df, feature):
    """
    Imputes missing values in a feature using Random Forest and returns imputation metrics.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the feature to impute.
    feature (str): The name of the feature to impute.

    Returns:
    tuple: (pd.DataFrame, dict) - The DataFrame with imputed feature and metrics.
    """
    # Separate the rows with and without missing values in the target feature
    df_missing = df[df[feature].isnull()]
    df_not_missing = df[~df[feature].isnull()]

    # If there are no missing values, return the original DataFrame with empty metrics
    if df_missing.empty:
        return df, {}

    # Determine if the feature is categorical or numerical
    if df[feature].dtype == 'object':
        # Categorical feature
        le = LabelEncoder()
        df_not_missing[feature] = le.fit_transform(df_not_missing[feature])

        # Prepare the data
        X = pd.get_dummies(df_not_missing.drop(columns=[feature]))
        y = df_not_missing[feature]

        # Split the data to evaluate the model
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Fit the Random Forest Classifier
        rf = RandomForestClassifier()
        rf.fit(X_train, y_train)

        # Predict on test set to get the accuracy
        y_pred = rf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Prepare the data for prediction (rows with missing values)
        X_missing = pd.get_dummies(df_missing.drop(columns=[feature]))
        X_missing = X_missing.reindex(columns=X.columns, fill_value=0)

        # Predict the missing values
        predicted = rf.predict(X_missing)
        df_missing[feature] = le.inverse_transform(predicted)

        # Combine the DataFrame with missing and non-missing parts
        df_not_missing[feature] = le.inverse_transform(df_not_missing[feature])
        df_imputed = pd.concat([df_not_missing, df_missing]).sort_index()

        # Metrics
        metrics = {
            'accuracy': accuracy
        }

    else:
        # Numerical feature
        # Prepare the data
        X = pd.get_dummies(df_not_missing.drop(columns=[feature]))
        y = df_not_missing[feature]

        # Split the data to evaluate the model
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Fit the Random Forest Regressor
        rf = RandomForestRegressor()
        rf.fit(X_train, y_train)

        # Predict on test set to get the mean squared error
        y_pred = rf.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Prepare the data for prediction (rows with missing values)
        X_missing = pd.get_dummies(df_missing.drop(columns=[feature]))
        X_missing = X_missing.reindex(columns=X.columns, fill_value=0)

        # Predict the missing values
        predicted = rf.predict(X_missing)
        df_missing[feature] = predicted

        # Combine the DataFrame with missing and non-missing parts
        df_imputed = pd.concat([df_not_missing, df_missing]).sort_index()

        # Metrics
        metrics = {
            'mean_squared_error': mse
        }

    return df_imputed, metrics

def impute_all_features(df):
    """
    Imputes missing values for all features in the DataFrame using Random Forest and returns imputed DataFrame and metrics.

    Parameters:
    df (pd.DataFrame): The DataFrame containing features to impute.

    Returns:
    tuple: (pd.DataFrame, dict) - The DataFrame with imputed features and metrics for each feature.
    """
    imputed_dfs = df.copy()
    all_metrics = {}

    for feature in df.columns:
        if df[feature].isnull().any():
            print(f"Before imputation of feature '{feature}':")
            print(imputed_dfs)
            imputed_dfs, metrics = impute_and_evaluate(imputed_dfs, feature)
            print(f"After imputation of feature '{feature}':")
            print(imputed_dfs)
            all_metrics[feature] = metrics

    return imputed_dfs, all_metrics


In [92]:
df = pd.read_csv('heart_disease_uci.csv')

# Impute missing values for all features
df_imputed_all, metrics_all = impute_all_features(df)

print("Final imputed DataFrame:")
print(df_imputed_all)
print("Metrics for all features:", metrics_all)

Final imputed DataFrame:
      id  age     sex        dataset               cp  trestbps   chol    fbs  \
0      1   63    Male      Cleveland   typical angina    145.00  233.0   True   
1      2   67    Male      Cleveland     asymptomatic    160.00  286.0  False   
2      3   67    Male      Cleveland     asymptomatic    120.00  229.0  False   
3      4   37    Male      Cleveland      non-anginal    130.00  250.0  False   
4      5   41  Female      Cleveland  atypical angina    130.00  204.0  False   
..   ...  ...     ...            ...              ...       ...    ...    ...   
915  916   54  Female  VA Long Beach     asymptomatic    127.00  333.0   True   
916  917   62    Male  VA Long Beach   typical angina    135.38  139.0  False   
917  918   55    Male  VA Long Beach     asymptomatic    122.00  223.0   True   
918  919   58    Male  VA Long Beach     asymptomatic    148.59  385.0   True   
919  920   62    Male  VA Long Beach  atypical angina    120.00  254.0  False   

  

In [97]:
df.isnull().sum().sort_values(ascending=False)

ca          611
thal        486
slope       309
fbs          90
oldpeak      62
trestbps     59
thalch       55
exang        55
chol         30
restecg       2
id            0
age           0
sex           0
dataset       0
cp            0
num           0
dtype: int64

In [94]:
df.isnull().sum()[df.isnull().sum() > 0].index.tolist()

['trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalch',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal']

In [93]:
df_imputed_all.isnull().sum().sort_values(ascending=False)

id          0
age         0
sex         0
dataset     0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalch      0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
num         0
dtype: int64

In [96]:
df_imputed_all.isnull().sum()[df_imputed_all.isnull().sum() > 0].index.tolist()

[]

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB
